In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# scraping kinopoisk, top250 films

In [3]:
driver = webdriver.Firefox()

In [4]:
driver.get('https://www.kinopoisk.ru/lists/movies/top250/?utm_referrer=www.google.com')

In [5]:
top250 = []
for x in range(1,6):
    driver.get(f'https://www.kinopoisk.ru/lists/movies/top250/?utm_referrer=www.google.com&page={x}')
    time.sleep(3)
    films = driver.find_elements(By.CLASS_NAME,'styles_root__ti07r')
    for film in films:
        title = film.find_element(By.CLASS_NAME,'base-movie-main-info_mainInfo__ZL_u3').text
        main_info1 = film.find_element(By.CLASS_NAME,'desktop-list-main-info_secondaryTitleSlot__mc0mI').text.replace('\n','')
        main_info2 = film.find_elements(By.CLASS_NAME,'desktop-list-main-info_additionalInfo__Hqzof')[0].text
        main_info3 = film.find_elements(By.CLASS_NAME,'desktop-list-main-info_additionalInfo__Hqzof')[1].text
        top250.append([title,main_info1,main_info2,main_info3])

In [14]:
pd.DataFrame(top250,columns=['title','main_info1','main_info2','main_info3']).to_csv('top250.csv',index=False)

In [3]:
df = pd.read_csv('top250.csv')

# Data cleaning

In [4]:
df.head()

,title,main_info1,main_info2,main_info3
0,Зеленая миля,"The Green Mile, 1999, 189 мин.",США • драма Режиссёр: Фрэнк Дарабонт,"В ролях: Том Хэнкс, Дэвид Морс"
1,Побег из Шоушенка,"The Shawshank Redemption, 1994, 142 мин.",США • драма Режиссёр: Фрэнк Дарабонт,"В ролях: Тим Роббинс, Морган Фриман"
2,Форрест Гамп,"Forrest Gump, 1994, 142 мин.",США • драма Режиссёр: Роберт Земекис,"В ролях: Том Хэнкс, Робин Райт"
3,1+1,"Intouchables, 2011, 112 мин.",Франция • драма Режиссёр: Оливье Накаш,"В ролях: Франсуа Клюзе, Омар Си"
4,Список Шиндлера,"Schindler's List, 1993, 195 мин.",США • драма Режиссёр: Стивен Спилберг,"В ролях: Лиам Нисон, Бен Кингсли"


split main_info1 by comma

In [5]:
df['main_info1'] = df['main_info1'].str.split(', ')

create column main_actors from main_info3

In [6]:
df['main_actors'] = df['main_info3'].str.replace('В ролях: ','')

In [7]:
df = df.drop('main_info3',axis=1)

create subcolumns: country, genre, producer from main_info2

In [8]:
df[['country','genre+producer']] = df['main_info2'].str.split('•',expand=True).rename({0:'country',1:'genre+producer'},axis=1)

In [9]:
df = df.drop('main_info2',axis=1)

In [10]:
df[['genre','producer']] = df['genre+producer'].str.split('  Режиссёр: ',expand=True)

In [11]:
df = df.drop('genre+producer',axis=1)

In [31]:
df

,title,main_actors,country,genre,producer,year,time_min
0,Зеленая миля,"Том Хэнкс, Дэвид Морс",США,драма,Фрэнк Дарабонт,1999,189
1,Побег из Шоушенка,"Тим Роббинс, Морган Фриман",США,драма,Фрэнк Дарабонт,1994,142
2,Форрест Гамп,"Том Хэнкс, Робин Райт",США,драма,Роберт Земекис,1994,142
3,1+1,"Франсуа Клюзе, Омар Си",Франция,драма,Оливье Накаш,2011,112
4,Список Шиндлера,"Лиам Нисон, Бен Кингсли",США,драма,Стивен Спилберг,1993,195
...,...,...,...,...,...,...,...
245,Берегись автомобиля,"Иннокентий Смоктуновский, Олег Ефремов",СССР,криминал,Эльдар Рязанов,1966,94
246,"Ирония судьбы, или С легким паром!","Андрей Мягков, Барбара Брыльска",СССР,мелодрама,Эльдар Рязанов,1975,184
247,Реквием по мечте,"Джаред Лето, Марлон Уайанс",США,драма,Даррен Аронофски,2000,102
248,Мумия,"Брендан Фрейзер, Рэйчел Вайс",США,фэнтези,Стивен Соммерс,1999,124


create subcolumns: time_min, year from main_info1

In [13]:
time_min = []
for x in range(len(df)):
    time_min.append(df['main_info1'][x][-1])

In [14]:
year = []
for x in range(len(df)):
    year.append(df['main_info1'][x][-2])

In [15]:
df['year'] = year
df['time_min'] = time_min

In [16]:
df['time_min'] = df['time_min'].str.replace(' мин.','')

columns time_min, year to numeric

In [17]:
df['time_min'] = pd.to_numeric(df['time_min'])
df['year'] = pd.to_numeric(df['year'])

In [24]:
df = df.drop('main_info1',axis=1)

# Small eda

rows,columns

In [18]:
df.shape

(250, 8)

duplicates

In [25]:
df.duplicated().mean()

0.0

nan values

In [26]:
df.isna().mean()

title          0.0
main_actors    0.0
country        0.0
genre          0.0
producer       0.0
year           0.0
time_min       0.0
dtype: float64

describe for numeric columns (mean,median,std,min,max,procentile)

In [28]:
df.describe()

,year,time_min
count,250.000000,250.000000
mean,1997.204000,122.148000
std,17.119541,29.648647
min,1939.000000,59.000000
25%,1986.250000,97.250000
50%,2001.000000,119.000000
75%,2010.750000,139.750000
max,2023.000000,222.000000


## top 5 country

In [ ]:
df['country'].value_counts().head(5)

country
США                110
СССР                55
Россия              17
Великобритания      15
Япония              14
Name: count, dtype: int64

## top 5 genre

In [30]:
df['genre'].value_counts().head(5)

genre
 драма         68
 мультфильм    33
 фэнтези       23
 мелодрама     19
 фантастика    18
Name: count, dtype: int64

## top 5 producers

In [35]:
df['producer'].value_counts().head()

producer
Хаяо Миядзаки      9
Леонид Гайдай      6
Крис Коламбус      6
Эльдар Рязанов     6
Кристофер Нолан    6
Name: count, dtype: int64